### **Prerequisits**

If you haven't installed Azure Data Studio (or ADS) please do so, as it is the client application that you can use to fire SQL queries against the server. You can download it from Microsoft [here on their page](https://docs.microsoft.com/en-us/sql/azure-data-studio/download-azure-data-studio?view=sql-server-2017).


### **PSRC's Data Wiki**

Did you know PSRC's Data Department has a [Wiki](http://aws-linux/mediawiki/index.php/Main_Page) for staff to use as a reference?  You will need to be connected to the VPN in order to access it.  Once you're in, you'll find links to lots of topics, but our focus for this lesson will be on the Elmer documentation.  You can find the link under the _Data Collection And Analysis_ header under _Projects_, or just use [this direct link](http://aws-linux/mediawiki/index.php/Elmer).

Near the [top of the page](http://aws-linux/mediawiki/index.php/Elmer#Elmer_Lives) you'll see a list of available datasets. We will be focusing on the [Census Summary Data](http://aws-linux/mediawiki/index.php/Elmer_Census_Summary_Data#Datasets_Available) for this lesson. That page has more technical information than we need to discuss today, but for the moment just note the _Datasets Available_ section at the top of the page, which provides an overview of the Census summary data available in Elmer.  It provides information about which Census tables are available in Elmer and at which geographies.  

We haven't brough in the entirety of the Census Bureau's data into Elmer -- there's just too much there -- but we have tried to bring in the data sets that people have asked Neil about in the past.  If you see a table that you would like included, please let me know and I will import it.

### **Connect to Elmer**


Assuming Azure Data Studio was successfully installed, we now have to connect ADS to the _Elmer_ database.  
1. Make sure the _Connections_ icon is selected on the far left top of ADS.  It should be just above the icon that looks like a magnifying glass.  
1. In the Connections panel, click _SERVERS_.
1. You should now see three icons near the top of the _Connections_ panel.  Click the _New Connection_ icon.
1. You should now be prompted for information about the server you want to connect to.  Enter the following values:
  * Server: AWS-PROD-SQL\Sockeye
  * Authentication Type: Windows Authentication
  * Database: Elmer


These instructions are also listed in the [connection instructions on the wiki](http://aws-linux/mediawiki/index.php/Elmer_Connection_Configuration#Connecting_via_Azure_Data_Studio). 



### **Some Important Points About Elmer Structure**

- Elmer's Census data is stored in large and inter-related tables (essentially grids of data), but we won't be accessing those tables directly. Instead, we will access the data through functions and stored procedures which, in turn, read from the tables and present the data to us.
- All Elmer's Census-related tables, functions and stored procedures must be prefixed with the text "`census.`"

### **The `census_table()` Function**

As mentioned above, Elmer's Census data are stored in several large inter-related tables.   We as staff don't need to learn the complexities of these underlying data structures because we have several intermediaries that do the work for us.  These are essentially canned queries that we can use to retrieve the data we want in a format that we can use.  The first of these is the `census_table()` function, which is documented on the wiki [here](http://aws-linux/mediawiki/index.php/Census.census_table_(table-valued_function).

In order to use the `census_table()` function we need to pass it four arguments:

- **`census_year`**: This is the Census year for the data you are interested in. At this time, the most recent data is for Census year 2019.
- **`geography_type`**: The geographic summary level at which you want to see the data. Common options are "Tract" and "Block Group" (for ACS 5-year data) and  "County" and "MSA" for ACS 1-year data.
- **`census_product`**: Elmer currently allows only two options here: "ACS1" and "ACS5", for ACS 1-year and 5-year estimates, respectively.
- **`census_table_code`**: The alphanumeric code for the Census table as published by the Census Bureau (e.g., "B01002", _Median Age By Sex_).

An example call to this function, with its four arguments, would look something like this:

```
census.census_table(2019, 'County', 'acs1', 'B01002')
```

### **Use `census_table()` in a query**

  
Under the hood, the `census_table()` function queries the underlying tables and produces a dataset that we can interact with as if it were its own, static table.  And like all tables, we need to query it in order to see the data inside it.  We do this through the `SELECT` statement, which tells the database to present us with the contents of the table.  

The `SELECT` clause is typically followed by a listing of exactly which columns we want returned from the table, but in our example here we use the asterisk which is short-hand for "all the columns":

In [ ]:
SELECT *
FROM census.census_table(2019, 'County', 'acs1', 'B01002')

As you can see, the function returns a table that might be different from what a lot of us are used to in working with Census data. That is, instead of a matrix with rows representing geographies and each variable existing in its own column, we have a long but narrow table with each row representing a distinct combination of variable and geography. Each of these combinations has a `estimate` and a `margin_of_error`.

This table format can be useful for consumption by dashboards in Tableau and other similar products, though it might not be the best for other uses like joining to shapefiles (we'll get to a better option for those in a bit).

The `census_table()` function always produces a table with these nine columns, regardless of how many variables there are the the Census table. For example, running it against the Census's Data Profile tables, which can contain hundreds of variables, result in a nine-column `census_table` with many hundred rows.


### **SELECT Specific Columns**

What if we don't want all nine columns returned to us?  Instead of using the asterisk, we can just list the column names we want:

In [ ]:
SELECT variable_name, variable_label, geography_name, estimate
FROM census.census_table(2019, 'County', 'acs1', 'B01002')

### **Filtering With WHERE**

What if we were interested in just the jurisdictions in Kitsap County?  We can limit the rows that the query returns with a **`WHERE`** clause:

In [ ]:
SELECT variable_name, variable_label, geography_name, estimate
FROM census.census_table(2019, 'County', 'acs1', 'B01002')
WHERE variable_name = 'B01002_001'

### **Sorting with ORDER BY**

We can use an ORDER BY clause to order our results

In [ ]:
SELECT variable_name, variable_label, geography_name, estimate
FROM census.census_table(2019, 'County', 'acs1', 'B01002')
WHERE variable_name = 'B01002_001'
ORDER BY geography_name

Now we have a result set with one row per county, with each _estimate_ displaying the meadian age for the total population.  

### **The `census_table_pivoted` Stored Procedure**

But what if we want all the variables for the Census table, but want them in their own columns? That is, can we do something akin to making a pivot table in Excel, so that each variable gets its own column?  We can do that, but the syntax is a bit different.  Instead of running a query with our handy SELECT statement, we need to run a _stored procedure_ called `census_table_pivoted`. This is essentially a small program that can be executed within SQL Server.  We don't have to wrap it in a `SELECT` query, but the downside of that is that we can't select only specific columns, can't filter the results with the `WHERE` clause, and can't sort them with the `ORDER BY` clause.  



As with the `census_table()` function, this stored procedure requires us to give it the same four argements `@census_year`, `@geography_type`, `@census_product` and `@census_table_code`. Unlike the function, the stored procedure does not require those to be enclosed within parentheses. We can run it by prefacing it with the **EXECUTE** command:

In [ ]:
EXECUTE census.census_table_pivoted 2019, 'County', 'ACS1', 'B01002'

As you can see, we get one row per geographical entity, with each variable represented as a distinct column.

### **Get More Informative Column Names with `@decode_col_names`**

By default, the column names that the stored procedure returns are the full English-language labels for the variables.  This might not be what you want, so it's possible to turn the labels off by passing in an additional optional argument **`@decode_col_names`** after the `@census_table_code`.  By setting this to 0 instead of 1 (which is the default) the variable headers will be displayed as the Census variable name (i.e., the codes) instead of the labels:

In [ ]:
EXECUTE census.census_table_pivoted 2019, 'County', 'ACS1', 'B01002', @decode_col_names = 0

### **Add Margins of Error with `@include_moe`**

The margin of errors for each estimate can be displayed by passing in the optional argument **`@include_moe`**.  The default value (0) causes just the estimates to be displayed, but if we set this argument to 1 then we get the margins of error just to the right of each estimate:

In [ ]:
EXECUTE census.census_table_pivoted 2019, 'County', 'ACS1', 'B01002', @include_moe = 1